In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from action import Action
from PDDL import PDDL_Parser
import sys, pprint
from collections import OrderedDict
import re, copy
import itertools
import z3
from skill_classes import EffectTypePDDL
from utils import product_dict, nested_list_replace

In [3]:
zeno_dom = "examples/zeno/zeno.pddl"
zeno_prob = "examples/zeno/pb1.pddl"

In [4]:
# domain, problem = dinner_dom, dinner_prob
domain, problem = zeno_dom, zeno_prob

parser = PDDL_Parser()
print('----------------------------')
# pprint.pprint(parser.scan_tokens(problem))
print('----------------------------')
parser.parse_domain(domain)
parser.parse_problem(problem)
print('Domain name: ' + parser.domain_name)
print(f'~~~Types~~~')
for t in parser.types: print(t)
print(f'~~~Type Hierarchy~~~')
for k,v in parser.type_hierarchy.items():
    print(f"{k}: {v}")
print(f'~~~Predicates~~~')
for nm, args in parser.predicates.items():
    print(f"{nm}({args}")
#     print("")
print('----------------------------')
print(f'~~~Functions~~~')
for nm, args in parser.functions.items():
    print(f"{nm}({args}")
#     print("")
print('----------------------------')
print('~~~Actions~~~')
for act in parser.actions:
    print(act)
    # print("effect types:")
    # for ef in action2EffectTypes(act): print(f"\t{ef}")
    print("\n")
print('----------------------------')
print('Problem name: ' + parser.problem_name)
print('Objects: ' + str(parser.objects))
print('State: ' + str(parser.state))
print('Positive goals: ' + str(parser.positive_goals))
print('Negative goals: ' + str(parser.negative_goals))

----------------------------
----------------------------
['locatable city - object', 'aircraft person - locatable']
Domain name: zenotravel
~~~Types~~~
aircraft
city
locatable
object
person
~~~Type Hierarchy~~~
aircraft: []
city: []
locatable: ['aircraft', 'person']
object: ['city', 'locatable']
person: []
~~~Predicates~~~
located(OrderedDict([('?x', 'locatable'), ('?c', 'city')])
in(OrderedDict([('?p', 'person'), ('?a', 'aircraft')])
----------------------------
~~~Functions~~~
fuel(OrderedDict([('?a', 'aircraft')])
distance(OrderedDict([('?c1', 'city'), ('?c2', 'city')])
slow-burn(OrderedDict([('?a', 'aircraft')])
fast-burn(OrderedDict([('?a', 'aircraft')])
capacity(OrderedDict([('?a', 'aircraft')])
total-fuel-used(OrderedDict()
onboard(OrderedDict([('?a', 'aircraft')])
zoom-limit(OrderedDict([('?a', 'aircraft')])
----------------------------
~~~Actions~~~
action: board
  parameters: [['?p', 'person'], ['?a', 'aircraft'], ['?c', 'city']]
  positive_preconditions: [['located', '?p', 

In [5]:
action_groundings = [parser.get_action_groundings(a) for a in parser.actions]
print("\n".join([str(x) for x in action_groundings[0]]))

action: board
  parameters: [['person1', 'person'], ['plane1', 'aircraft'], ['city0', 'city']]
  positive_preconditions: [['located', 'person1', 'city0'], ['located', 'plane1', 'city0']]
  negative_preconditions: []
  add_effects: [['in', 'person1', 'plane1'], ['increase', ['onboard', 'plane1'], '1']]
  del_effects: [['located', 'person1', 'city0']]

action: board
  parameters: [['person1', 'person'], ['plane1', 'aircraft'], ['city1', 'city']]
  positive_preconditions: [['located', 'person1', 'city1'], ['located', 'plane1', 'city1']]
  negative_preconditions: []
  add_effects: [['in', 'person1', 'plane1'], ['increase', ['onboard', 'plane1'], '1']]
  del_effects: [['located', 'person1', 'city1']]

action: board
  parameters: [['person1', 'person'], ['plane1', 'aircraft'], ['city2', 'city']]
  positive_preconditions: [['located', 'person1', 'city2'], ['located', 'plane1', 'city2']]
  negative_preconditions: []
  add_effects: [['in', 'person1', 'plane1'], ['increase', ['onboard', 'plane1'

In [6]:
def list_is_flat(l):
    for x in l: 
        if isinstance(x, list):
            return False
    return True

In [7]:
def make_z3Bool_from_list(expr_list):
    return_str = precond_list[0] + '('
    for obj in precond_list[1:]:
        return_str += obj + ', '
    return_str = return_str[:-2]
    return_str += ')'
    return_expr = z3.Bool(return_str)
    return return_expr

In [8]:
for a_id, action in enumerate(parser.actions):
    grounding = action_groundings[a_id]    

In [9]:
print(parser.actions[0])

action: board
  parameters: [['?p', 'person'], ['?a', 'aircraft'], ['?c', 'city']]
  positive_preconditions: [['located', '?p', '?c'], ['located', '?a', '?c']]
  negative_preconditions: []
  add_effects: [['in', '?p', '?a'], ['increase', ['onboard', '?a'], '1']]
  del_effects: [['located', '?p', '?c']]



In [10]:
for k, v in parser.predicates.items():
    print(k,v)

located OrderedDict([('?x', 'locatable'), ('?c', 'city')])
in OrderedDict([('?p', 'person'), ('?a', 'aircraft')])


## Make z3 vars

Make all z3 bools

In [11]:
def make_z3_atoms(things_dict, z3_class, str2var = None):
    """
    things_dict should be parser.predicates or parser.functions
    z3_class should be z3.Bool or z3.Int
    str2var is the dictionary you want to update. If none, a new OrderedDict is created
    """
    if str2var is None:
        str2var = OrderedDict()
    for p_name, p_args in things_dict.items():
        # print(p_name)
        groundings = product_dict(**OrderedDict([(varnm, parser.get_objects_of_type(vartype)) for (varnm, vartype) in p_args.items()]))
        for x in groundings:
            # print(x)
            s = p_name + "(" + ", ".join(x.values()) + ")"
            # print(s)
            grounded_predicate = z3_class(s)
            assert s not in str2var.keys(), f"{s}: {str2var[s]}, {grounded_predicate}"
            str2var[s] = grounded_predicate
    return str2var

In [12]:
str2var = OrderedDict()
make_z3_atoms(parser.predicates, z3.Bool, str2var);
make_z3_atoms(parser.functions, z3.Int, str2var);

In [13]:
for k, v in str2var.items():
    print(k,v, type(v))

located(plane1, city0) located(plane1, city0) <class 'z3.z3.BoolRef'>
located(plane1, city1) located(plane1, city1) <class 'z3.z3.BoolRef'>
located(plane1, city2) located(plane1, city2) <class 'z3.z3.BoolRef'>
located(person1, city0) located(person1, city0) <class 'z3.z3.BoolRef'>
located(person1, city1) located(person1, city1) <class 'z3.z3.BoolRef'>
located(person1, city2) located(person1, city2) <class 'z3.z3.BoolRef'>
located(person2, city0) located(person2, city0) <class 'z3.z3.BoolRef'>
located(person2, city1) located(person2, city1) <class 'z3.z3.BoolRef'>
located(person2, city2) located(person2, city2) <class 'z3.z3.BoolRef'>
located(person3, city0) located(person3, city0) <class 'z3.z3.BoolRef'>
located(person3, city1) located(person3, city1) <class 'z3.z3.BoolRef'>
located(person3, city2) located(person3, city2) <class 'z3.z3.BoolRef'>
in(person1, plane1) in(person1, plane1) <class 'z3.z3.BoolRef'>
in(person2, plane1) in(person2, plane1) <class 'z3.z3.BoolRef'>
in(person3, pl

In [14]:
a = action_groundings[0][0]; print(a)

action: board
  parameters: [['person1', 'person'], ['plane1', 'aircraft'], ['city0', 'city']]
  positive_preconditions: [['located', 'person1', 'city0'], ['located', 'plane1', 'city0']]
  negative_preconditions: []
  add_effects: [['in', 'person1', 'plane1'], ['increase', ['onboard', 'plane1'], '1']]
  del_effects: [['located', 'person1', 'city0']]



In [15]:
# This works for bool pvar preconditions, not more complex preconditions
z3_preconditions = []
for p in a.positive_preconditions:
    s = p[0] + "(" + ", ".join(p[1:]) + ")"
    z3_preconditions.append(str2var[s])
for p in a.negative_preconditions:
    s = p[0] + "(" + ", ".join(p[1:]) + ")"
    z3_preconditions.append(z3.Not(str2var[s]))
print(z3_preconditions)

[located(person1, city0), located(plane1, city0)]


In [16]:
# Will use this in AST type parser for complex preconditions. Not yet in use. Maybe useful to look at how we traversed the ast in the RDDL parser
str2comparator = {
    "<": lambda a, b: a < b,
    "<=": lambda a, b: a <= b,
    ">": lambda a, b: a > b,
    ">=": lambda a, b: a >= b,
    "=": lambda a, b: a == b
}

In [17]:
a = list(parser.actions)[2]
for p in a.positive_preconditions: print(p)

['located', '?a', '?c1']
['>=', ['fuel', '?a'], ['*', ['distance', '?c1', '?c2'], ['slow-burn', '?a']]]
